<a href="https://colab.research.google.com/github/i-m-sankha16/Exploratory-Data-Analysis/blob/master/Sales_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from __future__ import division

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

Using TensorFlow backend.


In [5]:

df_sales = pd.read_csv('/content/drive/My Drive/train_sales.csv')

In [6]:
df_sales.shape

(913000, 4)

In [7]:
df_sales.head(10)


,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
5,2013-01-06,1,1,12
6,2013-01-07,1,1,10
7,2013-01-08,1,1,9
8,2013-01-09,1,1,12
9,2013-01-10,1,1,9


In [8]:
df_sales['date'] = pd.to_datetime(df_sales['date'])

In [9]:
#represent month in date field as its first day
df_sales['date'] = df_sales['date'].dt.year.astype('str') + '-' + df_sales['date'].dt.month.astype('str') + '-01'
df_sales['date'] = pd.to_datetime(df_sales['date'])

In [10]:
#groupby date and sum the sales
df_sales = df_sales.groupby('date').sales.sum().reset_index()

In [11]:
df_sales.head()

,date,sales
0,2013-01-01,454904
1,2013-02-01,459417
2,2013-03-01,617382
3,2013-04-01,682274
4,2013-05-01,763242


In [12]:
import plotly.graph_objects as go
#plot monthly sales
plot_data = [
    go.Scatter(
        x=df_sales['date'],
        y=df_sales['sales'],
    )
]

plot_layout = go.Layout(
        title='Montly Sales'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
#pyoff.iplot(fig)

In [13]:
fig.show()

In [14]:
#create a new dataframe to model the difference
df_diff = df_sales.copy()

In [15]:
#add previous sales to the next row
df_diff['prev_sales'] = df_diff['sales'].shift(1)

In [16]:
df_diff.head()


,date,sales,prev_sales
0,2013-01-01,454904,NaN
1,2013-02-01,459417,454904.0
2,2013-03-01,617382,459417.0
3,2013-04-01,682274,617382.0
4,2013-05-01,763242,682274.0


In [17]:
#drop the null values and calculate the difference
df_diff = df_diff.dropna()

In [18]:
df_diff['diff'] = (df_diff['sales'] - df_diff['prev_sales'])

In [19]:
df_diff.head(10)

,date,sales,prev_sales,diff
1,2013-02-01,459417,454904.0,4513.0
2,2013-03-01,617382,459417.0,157965.0
3,2013-04-01,682274,617382.0,64892.0
4,2013-05-01,763242,682274.0,80968.0
5,2013-06-01,795597,763242.0,32355.0
6,2013-07-01,855922,795597.0,60325.0
7,2013-08-01,766761,855922.0,-89161.0
8,2013-09-01,689907,766761.0,-76854.0
9,2013-10-01,656587,689907.0,-33320.0
10,2013-11-01,692643,656587.0,36056.0


In [20]:
#plot sales diff
plot_data = [
    go.Scatter(
        x=df_diff['date'],
        y=df_diff['diff'],
    )
]

plot_layout = go.Layout(
        title='Montly Sales Diff'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
#pyoff.iplot(fig)

In [21]:
fig.show()

In [22]:
#create new dataframe from transformation from time series to supervised
df_supervised = df_diff.drop(['prev_sales'],axis=1)

In [23]:
#adding lags
for inc in range(1,13):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)

In [24]:
df_supervised.head(10)

,date,sales,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
1,2013-02-01,459417,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-03-01,617382,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-04-01,682274,64892.0,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-05-01,763242,80968.0,64892.0,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013-06-01,795597,32355.0,80968.0,64892.0,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013-07-01,855922,60325.0,32355.0,80968.0,64892.0,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013-08-01,766761,-89161.0,60325.0,32355.0,80968.0,64892.0,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN,NaN
8,2013-09-01,689907,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN
9,2013-10-01,656587,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0,157965.0,4513.0,NaN,NaN,NaN,NaN
10,2013-11-01,692643,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0,157965.0,4513.0,NaN,NaN,NaN


In [25]:
df_supervised.tail(6)

,date,sales,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
54,2017-07-01,1171393,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0,27811.0,-33194.0,-84663.0,-157224.0,116054.0
55,2017-08-01,1026403,-144990.0,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0,27811.0,-33194.0,-84663.0,-157224.0
56,2017-09-01,935263,-91140.0,-144990.0,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0,27811.0,-33194.0,-84663.0
57,2017-10-01,891160,-44103.0,-91140.0,-144990.0,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0,27811.0,-33194.0
58,2017-11-01,928837,37677.0,-44103.0,-91140.0,-144990.0,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0,27811.0
59,2017-12-01,695170,-233667.0,37677.0,-44103.0,-91140.0,-144990.0,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0


In [26]:
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)

In [27]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.02893426930900389


In [28]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.4406493613886947


In [29]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5 + lag_6 + lag_7 + lag_8 + lag_9 + lag_10 + lag_11 + lag_12', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.9795722233296558


In [30]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['sales','date'],axis=1)

In [31]:
#split train and test set
train_set, test_set = df_model[0:-6].values, df_model[-6:].values

In [32]:
df_model.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   diff    47 non-null     float64
 1   lag_1   47 non-null     float64
 2   lag_2   47 non-null     float64
 3   lag_3   47 non-null     float64
 4   lag_4   47 non-null     float64
 5   lag_5   47 non-null     float64
 6   lag_6   47 non-null     float64
 7   lag_7   47 non-null     float64
 8   lag_8   47 non-null     float64
 9   lag_9   47 non-null     float64
 10  lag_10  47 non-null     float64
 11  lag_11  47 non-null     float64
 12  lag_12  47 non-null     float64
dtypes: float64(13)
memory usage: 4.9 KB


In [33]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)

# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [34]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

In [35]:
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [36]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, nb_epoch=100, batch_size=1, verbose=1, shuffle=False)

Epoch 1/100
41/41 [==============================] - 2s 50ms/step - loss: 0.3063
Epoch 2/100
41/41 [==============================] - 0s 4ms/step - loss: 0.2550
Epoch 3/100
41/41 [==============================] - 0s 4ms/step - loss: 0.2221
Epoch 4/100
41/41 [==============================] - 0s 4ms/step - loss: 0.1969
Epoch 5/100
41/41 [==============================] - 0s 5ms/step - loss: 0.1769
Epoch 6/100
41/41 [==============================] - 0s 5ms/step - loss: 0.1603
Epoch 7/100
41/41 [==============================] - 0s 4ms/step - loss: 0.1464
Epoch 8/100
41/41 [==============================] - 0s 4ms/step - loss: 0.1344
Epoch 9/100
41/41 [==============================] - 0s 4ms/step - loss: 0.1240
Epoch 10/100
41/41 [==============================] - 0s 4ms/step - loss: 0.1147
Epoch 11/100
41/41 [==============================] - 0s 4ms/step - loss: 0.1064
Epoch 12/100
41/41 [==============================] - 0s 5ms/step - loss: 0.0989
Epoch 13/100
41/41 [================

In [37]:
y_pred = model.predict(X_test,batch_size=1)

In [38]:
y_pred

array([[ 0.5329441 ],
       [-0.68242717],
       [-0.44260636],
       [-0.07426498],
       [ 0.12972467],
       [-1.0644556 ]], dtype=float32)

In [39]:
y_test

array([[ 0.55964922],
       [-0.61313659],
       [-0.36228353],
       [-0.14316792],
       [ 0.23779333],
       [-1.02622661]])

In [40]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])

In [43]:
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    #print (np.concatenate([y_pred[index],X_test[index]],axis=1)
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))

In [44]:
pred_test_set[0]

array([[ 0.53294408,  0.26695937,  0.44344626,  0.60355899,  1.10628178,
         0.13866328, -0.10745675, -1.02635392,  0.24535439, -0.05787474,
        -0.31370458, -0.67437352,  0.68397168]])

In [45]:
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])

In [46]:
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

In [47]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df_sales[-7:].date)
act_sales = list(df_sales[-7:].sales)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['date'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)

In [48]:
df_result

,pred_value,date
0,1165660,2017-07-01
1,1011528,2017-08-01
2,918020,2017-09-01
3,905951,2017-10-01
4,905638,2017-11-01
5,686963,2017-12-01


In [49]:
df_sales.head()

,date,sales
0,2013-01-01,454904
1,2013-02-01,459417
2,2013-03-01,617382
3,2013-04-01,682274
4,2013-05-01,763242


In [50]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df_sales,df_result,on='date',how='left')

In [51]:
df_sales_pred


,date,sales,pred_value
0,2013-01-01,454904,NaN
1,2013-02-01,459417,NaN
2,2013-03-01,617382,NaN
3,2013-04-01,682274,NaN
4,2013-05-01,763242,NaN
5,2013-06-01,795597,NaN
6,2013-07-01,855922,NaN
7,2013-08-01,766761,NaN
8,2013-09-01,689907,NaN
9,2013-10-01,656587,NaN


In [52]:
#plot actual and predicted
plot_data = [
    go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['sales'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['pred_value'],
        name='predicted'
    )
    
]

plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
#pyoff.iplot(fig)

In [53]:
fig.show()

In [54]:
import pickle 
  
# Save the trained model as a pickle string. 
saved_model = pickle.dumps(model) 
  
# Load the pickled model 
model_from_pickle = pickle.loads(saved_model) 
  
# Use the loaded pickled model to make predictions 
model_from_pickle.predict(X_test)

array([[0.3135933],
       [0.3135933],
       [0.3135933],
       [0.3135933],
       [0.3135933],
       [0.3135933]], dtype=float32)